In [1]:
# import os
# import jsonlines

# subsets = ["train", "dev", "test"]
# langs = ["en", "vi"]

# for subset in subsets:
#     temp = {}
#     data = []
#     for lang in langs:
#         path = os.path.join("../data/PhoMT/tokenization",subset,f"{subset}.{lang}")
#         with open(path, "r", encoding='utf-8') as f:    
#             contents = f.readlines()
#         print(f"{path}:", len(contents), "lines")
#         for i in range(len(contents)):
#             line = contents[i].strip()
#             if line[-1:] == "\n":
#                 line = line[:-1]
#             if lang == "vi":
#                 contents[i] = "translate Vietnamese to English: " + line + " </s>"
#             else:
#                 contents[i] = line
#         temp[lang] = contents
#     for en, vi in zip(temp["en"], temp["vi"]):
#         data.append({
#             "en": en,
#             "vi": vi
#         })
#     with jsonlines.open(f'../data/PhoMT/tokenization/{subset}.jsonl', mode='w') as writer:        
#         writer.write_all(data)
# del temp
# del data


In [1]:
import wandb

wandb.init(project="vietai-machine-translation", entity="fantastic-four", name="vit5_chkp-30000")


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: huyhuy (fantastic-four). Use `wandb login --relogin` to force relogin


In [2]:
from datasets import load_dataset

dataset = load_dataset(
    "json", 
    data_files={
        "train":"../data/PhoMT/tokenization/train.jsonl",
        "dev":"../data/PhoMT/tokenization/dev.jsonl",
        "test":"../data/PhoMT/tokenization/test.jsonl"
    },
    streaming=True
)
dataset = dataset.with_format("torch")

Using custom data configuration default-961e8ec10ee50d7d


In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")
# model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base").to("cuda:1")

tokenizer = AutoTokenizer.from_pretrained("vit5-base-3/checkpoint-30000")
model = AutoModelForSeq2SeqLM.from_pretrained("vit5-base-3/checkpoint-30000").to("cuda:1")
# model.gradient_checkpointing_enable()
# model.use_cache = False
max_input_length = 128
max_target_length = 128

In [4]:
def preprocess_function(examples):
    inputs = examples["vi"] 
    targets =examples["en"] 
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["en", "vi"]
)
del dataset

In [5]:
import psutil

# Process.memory_info is expressed in bytes, so convert to megabytes
print(f"RAM used: {psutil.Process().memory_info().rss / (1024 * 1024):.2f} MB")

RAM used: 2607.81 MB


In [5]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

In [6]:
import numpy as np
import evaluate

metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [7]:
from transformers import Seq2SeqTrainingArguments

batch_train = 64
num_epochs = 2
args = Seq2SeqTrainingArguments(
    f"vit5-base-4",
    evaluation_strategy = 'steps',
    save_strategy="steps",
    logging_steps = 500,                   
    eval_steps = 5000, 
    save_steps=1000,
    learning_rate=0.001,
    per_device_eval_batch_size=128,
    save_total_limit=3,
    max_steps= 40000,
    predict_with_generate=True,
    fp16=True,
    half_precision_backend = "auto",
    report_to="wandb",  # enable logging to W&B
    # run_name="vit5-1",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=8
)



# 2977999 

In [8]:
from transformers.optimization import Adafactor, AdafactorSchedule

optim = Adafactor(
    model.parameters(), 
    scale_parameter=True, 
    relative_step=True, 
    warmup_init=True, 
    lr=None
)
lr_scheduler = AdafactorSchedule(optim)

In [9]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(optim, lr_scheduler)
)

max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend


In [12]:
# trainer.evaluate(max_length=max_target_length)

In [9]:
trainer.predict(tokenized_datasets["test"])

***** Running Prediction *****
  Num examples: Unknown
  Batch size = 256
/home/cotai/anaconda3/envs/vietai/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


PredictionOutput(predictions=array([[    0,  5559,  1827, ...,  6853,  2695,     5],
       [    0,  3222,    40, ...,  2165, 35807,  4768],
       [    0,  1746, 26159, ...,  1402, 12661,  6855],
       ...,
       [    0,  1263, 28186, ...,   940,  2273, 14961],
       [    0,  3600, 22214, ..., 32926,     5, 11196],
       [    0, 14284, 24770, ...,  2282,  2174,  2273]]), label_ids=array([[29332,  5559,  1827, ...,  -100,  -100,  -100],
       [ 5111,  4398,  3826, ...,  -100,  -100,  -100],
       [12131,    35,  7487, ...,  -100,  -100,  -100],
       ...,
       [ 1263, 28186,     5, ...,  -100,  -100,  -100],
       [25162,  8652, 35793, ...,  -100,  -100,  -100],
       [   26,  4019,  4829, ...,  -100,  -100,  -100]]), metrics={'test_loss': 1.4580744504928589, 'test_bleu': 14.853603979613457, 'test_runtime': 118.9247, 'test_samples_per_second': 161.035, 'test_steps_per_second': 0.631})

In [10]:
trainer.train("vit5-base-3/checkpoint-30000")


Loading model from vit5-base-3/checkpoint-30000.
***** Running training *****
  Num examples = 10240000
  Num Epochs = 9223372036854775807
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 8
  Total optimization steps = 40000
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 0
  Continuing training from global step 30000
  Will skip the first 0 epochs then the first 240000 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/240000 [00:00<?, ?it/s]

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/home/cotai/anaconda3/envs/vietai/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.evaluate(max_length=max_target_length)

***** Running Evaluation *****
  Num examples: Unknown
  Batch size = 256
/home/cotai/anaconda3/envs/vietai/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


{'eval_loss': 3.6363742351531982,
 'eval_bleu': 15.480611927582189,
 'eval_runtime': 183.5569,
 'eval_samples_per_second': 101.979,
 'eval_steps_per_second': 0.403}

In [11]:
sentence = "Các trưởng lão địa phương và giám thị xung quanh đang giúp đỡ và cung cấp về vật chất và tinh thần cho các anh chị bị ảnh hưởng trong thảm hoạ này ."
text =  "translate Vietnamese to English: " + sentence + " </s>"
encoding = tokenizer(text, padding=True, max_length=max_input_length, truncation=True, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to("cuda:1"), encoding["attention_mask"].to("cuda:1")
outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=max_target_length,
    # early_stopping=True,
    # do_sample=True,
    # num_beams=5,
    # num_return_sequences=1,
    # no_repeat_ngram_size=1,
    # remove_invalid_values=True,
)
for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(line)

Local elders and the neighborhood supervisors are helping and providing physical and mentality to the brothers that are affected by this disaster.


In [ ]:
a="Local elders and the circuit overseer are offering practical and spiritual support to those affected by this disaster ."